# 3. Wrapping a model into a class - the Single Particle Model

In the previous notebooks we learnt how to build models using PyBaMM. We implemented them directly in a notebook, which is very useful when building the model for the first time, but not so much if we want to use the model often or share it with other people. In these cases, it is better to wrap the model into a class. 

A [Python class](https://docs.python.org/3/tutorial/classes.html) is a "blueprint" to create an object. A Python object is a special variable that contain data (variables) and methods (functions) that can manipulate this data. In our case, the object will be a model, which we can then solve. If you are not familiar with Python classes it is worth learning about them with one of the many [online resources](https://www.w3schools.com/python/python_classes.asp), as in this notebook we will assume a basic knowledge of them.

The goal for this notebook is to write the SPM for a half-cell implemented in the [previous notebook](./2-a-pde-model.ipynb) as a class.

## Setting up the model

When defining the model as a class, it needs to inherit from the `pybamm.BaseModel` class. Then, the variables, parameters and equations are defined in the `__init__` method. For the half-cell SPM:

In [1]:
import pybamm

class BasicSPMHalfCell(pybamm.BaseModel):
    def __init__(self, name="Single Particle Model"):
        # Initialise the parent class, mainly to define the model name
        super().__init__(name)

        ######################
        # Variables
        ######################
        c_p = pybamm.Variable(
            "Positive particle concentration [mol.m-3]", domain="positive particle"
        )

        ######################
        # Parameters
        ######################
        # constant parameters
        D_p = pybamm.Parameter("Positive electrode diffusivity [m2.s-1]")
        R_p = pybamm.Parameter("Positive particle radius [m]")
        A = pybamm.Parameter("Electrode plate area [m2]")
        epsilon_s_p = pybamm.Parameter("Positive electrode active material volume fraction")
        a_p = 3 * epsilon_s_p / R_p  # assume spherical particles
        L_p = pybamm.Parameter("Positive electrode thickness [m]")
        T = pybamm.Parameter("Temperature [K]")
        c_p_0 = pybamm.Parameter("Initial concentration in positive electrode [mol.m-3]")
        c_p_max = pybamm.Parameter("Maximum concentration in positive electrode [mol.m-3]")
        k_p = pybamm.Parameter("Positive electrode reaction rate [m.s-1]")

        # function parameters
        I = pybamm.FunctionParameter("Current function [A]", {"Time [s]": pybamm.t})
        U_p = pybamm.FunctionParameter("Positive electrode OCV [V]", {"x_p": c_p / c_p_max})

        # constants
        R = pybamm.constants.R
        F = pybamm.constants.F

        ######################
        # Particle model
        ######################
        # equation
        N = -D_p * pybamm.grad(c_p)
        dcpdt = -pybamm.div(N)
        self.rhs = {c_p: dcpdt}

        # initial conditions
        self.initial_conditions = {c_p: c_p_0}

        # boundary conditions
        lbc = pybamm.Scalar(0)
        i = I / A
        rbc = i / (a_p * F * L_p * D_p)
        self.boundary_conditions = {c_p: {"left": (lbc, "Neumann"), "right": (rbc, "Neumann")}}

        ######################
        # (Some) variables
        ######################
        # define auxiliary expressions
        c_p_surf = pybamm.surf(c_p)
        j_p_0 = F * k_p * (c_p_surf * (c_p_max - c_p_surf)) ** 0.5
        V = pybamm.surf(U_p) - 2 * R * T / F * pybamm.arcsinh(i / (a_p * L_p * j_p_0))

        # define the model variables
        self.variables = {
            "Positive particle concentration [mol.m-3]": c_p,
            "Positive particle surface concentration [mol.m-3]": c_p_surf,
            "Voltage [V]": V,
        }

Now, we can generate an instance of the model by calling the class:

In [2]:
model = BasicSPMHalfCell()

### Default attributes (optional)

What we have done so far is enough to run the model, but there are some optional extra steps we can take that will make our lives easier going forward. When a model processed by the simulation class, it uses several additional objects (e.g. geometry, solver...) that in our previous notebook we had to manually pass as keyword arguments. Quite often, we want to repeatedly use the same values for these keyword arguments, so we can define their default values when defining the model. This means that, if one of the keyword arguments is not explicitly defined in the simulation, PyBaMM will use the default ones when running the simulation.

These default attributes are defined by methods in the class (e.g. `default_geometry`, `default_solver`...) and they always follow the same structure:

```python3
def default_attribute(self):
   # insert here intermediate steps to define default
   
   return default 
```

The default attributes need to be defined as class properties. A class property is an attribute that is computed dynamically and accessed like a regular attribute but allows additional logic to be executed when it is accessed. This is achieved using the `@property` decorator. In general we would just define them as additional class methods, but in order to not have to define the model again we can simply override the class inheriting from the previous one:

In [3]:
import numpy as np

class BasicSPMHalfCell(BasicSPMHalfCell):
    def __init__(self, name="Single Particle Model"):
        # Initialise the parent class to define the model
        super().__init__(name)

    @property
    def default_parameter_values(self):
        def nmc_LGM50_ocp_Chen2020(sto):
            u_eq = (
                -0.8090 * sto
                + 4.4875
                - 0.0428 * np.tanh(18.5138 * (sto - 0.5542))
                - 17.7326 * np.tanh(15.7890 * (sto - 0.3117))
                + 17.5842 * np.tanh(15.9308 * (sto - 0.3120))
            )

            return u_eq
        
        parameter_values = pybamm.ParameterValues(
            {
                "Positive electrode diffusivity [m2.s-1]": 4e-15,
                "Positive particle radius [m]": 5.22e-06,
                "Positive electrode active material volume fraction": 0.665,
                "Positive electrode thickness [m]": 7.56e-05,
                "Temperature [K]": 298.15,
                "Initial concentration in positive electrode [mol.m-3]": 17038.0,
                "Maximum concentration in positive electrode [mol.m-3]": 63104.0,
                "Positive electrode reaction rate [m.s-1]": 1.1e-9,
                "Current function [A]": 5,
                "Electrode plate area [m2]": 0.1,
                "Positive electrode OCV [V]": nmc_LGM50_ocp_Chen2020,
            }
        )

        return parameter_values

    @property
    def default_geometry(self):
        r = pybamm.SpatialVariable(
            "r", domain=["positive particle"], coord_sys="spherical polar"
        )
        R_p = pybamm.Parameter("Positive particle radius [m]")
        geometry = {"positive particle": {r: {"min": pybamm.Scalar(0), "max": R_p}}}

        return geometry

    @property
    def default_submesh_types(self):
        submesh_types = {"positive particle": pybamm.Uniform1DSubMesh}
        return submesh_types

    @property
    def default_var_pts(self):
        r = pybamm.SpatialVariable(
            "r", domain=["positive particle"], coord_sys="spherical polar"
        )
        var_pts = {r: 20}
        return var_pts

    @property
    def default_spatial_methods(self):
        spatial_methods = {"positive particle": pybamm.FiniteVolume()}
        return spatial_methods

    @property
    def default_solver(self):
        return pybamm.ScipySolver()

    @property
    def default_quick_plot_variables(self):
        return [
            "Voltage [V]",
            "Positive particle concentration [mol.m-3]",
            "Positive particle surface concentration [mol.m-3]",
        ]

## Solving the model

Now, we can just create a simulation specifying the model and PyBaMM will take the default values for everything else:

In [4]:
model = BasicSPMHalfCell()
sim = pybamm.Simulation(model)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

We obtain the same results as in the previous notebook, but we can solve the model in a much more compact notation. We can still override the defaults by passing the corresponding keyword arguments. For example, to override the parameter values:

In [5]:
new_parameter_values = model.default_parameter_values
new_parameter_values["Current function [A]"] = 2.5

model = BasicSPMHalfCell()
sim = pybamm.Simulation(model, parameter_values=new_parameter_values)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In this notebook we have shown how to wrap a PyBaMM model into a class and to define its default attributes, both of which make running the models a lot simpler and more compact. In the next notebook we will introduce some additional tricks to simplify the process, and we will build the Single Particle Model.

## References

The relevant papers for this notebook are:

In [6]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[3] Valentin Sulzer, Scott G. Marquis, Robert Timms, Martin Robinson, and S. Jon Chapman. Python Battery Mathematical Modelling (PyBaMM). Journal of Open Research Software, 9(1):14, 2021. doi:10.5334/jors.309.
[4] Pauli Virtanen, Ralf Gommers, Travis E. Oliphant, Matt Haberland, Tyler Reddy, David Cournapeau, Evgeni Burovski, Pearu Peterson, Warren Weckesser, Jonathan Bright, and others. SciPy 1.0: fundamental algorithms for scientific computing in Python. Na